In [19]:
import pandas as pd
import requests
import pprint
import json

In [29]:
url = "https://api.steampowered.com/ISteamApps/GetAppList/v0002/?format=json"
response = requests.get(url)
json_data = response.json()

gamesArray = json_data["applist"]["apps"]

gameDetailsUrl = "https://store.steampowered.com/api/appdetails?appids="
   

gamesIdArray = []

gamesInfoObject = {}

test = 0

for game in gamesArray:
  gamesIdArray.append(game["appid"])

for gameID in gamesIdArray:

  try:

    gameResponse = requests.get(f'{gameDetailsUrl}{gameID}')
    json_data_details = gameResponse.json() 

    if json_data_details[str(gameID)]['success'] == True:
      
      if json_data_details[str(gameID)]["data"]["type"] == "game":

        gamesInfoObject[gameID] = {"game_name": json_data_details[str(gameID)]["data"]["name"], 
                                  "is_free": json_data_details[str(gameID)]["data"]["is_free"],
                                  "genres": ", ".join(x["description"] for x in json_data_details[str(gameID)]["data"]["genres"]),
                                  "categories": ", ".join(x["description"] for x in json_data_details[str(gameID)]["data"]["categories"]),
                                  "release_date": json_data_details[str(gameID)]["data"]["release_date"]["date"],
                                  "price": json_data_details[str(gameID)]["data"]["price_overview"]["final_formatted"],
                                  "required_age": json_data_details[str(gameID)]["data"]["required_age"]
                                  }
        
  except:
    next      
  


In [30]:
import json

with open('output.json', 'w') as json_file:
    json.dump(gamesInfoObject, json_file, indent=4)



In [31]:
gamesInfoArray = []

for key, value in gamesInfoObject.items():
    gamesInfoArray.append({"game_id": key, **value})

In [34]:
games_df = pd.DataFrame(gamesInfoArray)

games_df.head()

,game_id,game_name,is_free,genres,categories,release_date,price,required_age
0,1744330,No More Heroes 3,False,Action,"Single-player, Steam Achievements, Full contro...","11 Oct, 2022",CDN$ 49.99,17
1,1743380,SCP: Doki Doki Anomaly,False,"Indie, RPG","Single-player, Steam Achievements","7 Oct, 2021",CDN$ 6.69,0
2,1742450,西游记2021,False,"Adventure, Casual, Indie, RPG, Strategy",Single-player,"30 Sep, 2021",CDN$ 11.49,0
3,1742470,My Newborn Life In A Harem,False,"Casual, RPG",Single-player,"29 Mar, 2022",CDN$ 14.49,0
4,1742500,My Dear Frankenstein -English Edition-,False,Adventure,Single-player,"30 Sep, 2021",CDN$ 5.69,0


In [41]:
games_df.to_csv('games_info.csv', index=False)

updatedGamesId = list(gamesInfoObject.keys())

print(len(updatedGamesId))



11968


In [46]:
reviewsObject = {}


for game in updatedGamesId:

  try:

    response_reviews = requests.get(f"https://store.steampowered.com/appreviews/{str(game)}?json=1&start_offset=0")
    json_datareviews = response_reviews.json()


    if json_datareviews['success'] == True:
      
      reviewsObject[game] = {"review_score_desc": json_datareviews['query_summary']["review_score_desc"],
                            "total_positive_reviews": json_datareviews['query_summary']["total_positive"],
                            "total_negative_reviews": json_datareviews['query_summary']["total_negative"],
                            "review_score": json_datareviews['query_summary']["review_score"]}
  except:
    next


  


In [47]:
with open('output2.json', 'w') as json_file:
    json.dump(reviewsObject, json_file, indent=4)

In [48]:
gamesReviewArray = []

for key, value in reviewsObject.items():
    gamesReviewArray.append({"game_id": key, **value})

In [49]:
gamesreview_df = pd.DataFrame(gamesReviewArray)

gamesreview_df.head()

,game_id,review_score_desc,total_positive_reviews,total_negative_reviews,review_score
0,1744330,Very Positive,232,46,8
1,1743380,Very Positive,51,4,8
2,1742450,No user reviews,0,0,0
3,1742470,Mixed,18,12,5
4,1742500,4 user reviews,4,0,0


In [52]:
game_info_and_review_df = pd.merge(games_df, gamesreview_df, on='game_id', how='inner')

game_info_and_review_df.head()

,game_id,game_name,is_free,genres,categories,release_date,price,required_age,review_score_desc,total_positive_reviews,total_negative_reviews,review_score
0,1744330,No More Heroes 3,False,Action,"Single-player, Steam Achievements, Full contro...","11 Oct, 2022",CDN$ 49.99,17,Very Positive,232,46,8
1,1743380,SCP: Doki Doki Anomaly,False,"Indie, RPG","Single-player, Steam Achievements","7 Oct, 2021",CDN$ 6.69,0,Very Positive,51,4,8
2,1742450,西游记2021,False,"Adventure, Casual, Indie, RPG, Strategy",Single-player,"30 Sep, 2021",CDN$ 11.49,0,No user reviews,0,0,0
3,1742470,My Newborn Life In A Harem,False,"Casual, RPG",Single-player,"29 Mar, 2022",CDN$ 14.49,0,Mixed,18,12,5
4,1742500,My Dear Frankenstein -English Edition-,False,Adventure,Single-player,"30 Sep, 2021",CDN$ 5.69,0,4 user reviews,4,0,0


In [17]:

gameOwnersObject = {}


for game in updatedGamesId:

  try:

    response_Owners = requests.get(f"https://steamspy.com/api.php?request=appdetails&appid={str(game)}")
    json_dataOwners = response_Owners.json()


    if response_Owners.status_code == 200:
      
      gameOwnersObject[game] = {"owners": json_dataOwners['owners']}

  except:
    next

In [20]:
with open('output3.json', 'w') as json_file:
    json.dump(gameOwnersObject, json_file, indent=4)

In [25]:
gameOwnersArray = []

for key, value in gameOwnersObject.items():
    gameOwnersArray.append({"game_id": key, **value})

In [28]:
gameOwners_df = pd.DataFrame(gameOwnersArray)
gameOwners_df.head()

,game_id,owners
0,1744330,"20,000 .. 50,000"
1,1743380,"0 .. 20,000"
2,1742450,"50,000 .. 100,000"
3,1742470,"0 .. 20,000"
4,1742500,"0 .. 20,000"


In [30]:
merged_df = pd.merge(game_info_and_review_df, gameOwners_df, on='game_id', how='inner')

In [32]:
merged_df.to_csv('games_info_and_reviews.csv', index=False)